# V16: Model competition (transit vs non-transit)

Compares multiple simple models against the folded signal: a transit-only model, a transit+sinusoid model (variability/systematics proxy), and an EB-like model.


In [ ]:
from pathlib import Path
import json
import sys

tutorial_dir = Path('docs/tutorials/tutorial_toi-5807-incremental').resolve()
sys.path.insert(0, str(tutorial_dir))

import toi5807_shared as sh

ds = sh.load_dataset()
lc = sh.stitch_pdcsap(ds)
depth_ppm, _ = sh.estimate_depth_ppm(lc)
candidate = sh.make_candidate(depth_ppm)

session = sh.make_session(stitched=lc, candidate=candidate, network=False, preset='extended')
r = session.run('V16')

print(
    json.dumps(
        {
            'status': r.status,
            'flags': list(r.flags),
            'metrics': r.metrics,
        },
        indent=2,
        sort_keys=True,
    )
)


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "flags": [
    "MODEL_PREFERS_NON_TRANSIT"
  ],
  "metrics": {
    "aic_eb_like": -1001048.857269804,
    "aic_transit_only": -1001020.4102233441,
    "aic_transit_sinusoid": -1001078.4687633369,
    "artifact_prior_combined_risk": 0.2401026934306569,
    "artifact_prior_period_alias_risk": 0.4802053868613138,
    "artifact_prior_scattered_light_risk": 0.0,
    "artifact_prior_sector_quality_risk": 0.0,
    "artifact_risk": 0.8,
    "bic_eb_like": -1001021.2297245271,
    "bic_transit_only": -1001011.2010415852,
    "bic_transit_sinusoid": -1001032.422854542,
    "model_competition_label": "SINUSOID",
    "winner": "transit_sinusoid",
    "winner_margin_bic": 11.19313001492992
  },
  "status": "ok"
}
```

</details>


In [ ]:
from pathlib import Path
import json

import matplotlib.pyplot as plt

from bittr_tess_vetter.plotting import plot_model_comparison

step_id = '18_v16_model_competition'
fname = 'V16_model_competition.png'

run_out_dir, docs_out_dir = sh.artifact_dirs(step_id=step_id)
run_path = run_out_dir / fname
docs_path = (docs_out_dir / fname) if docs_out_dir is not None else None

out = {'status': r.status, 'flags': list(r.flags)}

if r.status == 'ok':
    fig, ax = plt.subplots(figsize=(8, 5))
    plot_model_comparison(r, ax=ax)
    ax.set_title('V16: Model competition (transit vs non-transit)')
    fig.tight_layout()
    fig.savefig(run_path, dpi=150, bbox_inches='tight')
    if docs_path is not None:
        fig.savefig(docs_path, dpi=150, bbox_inches='tight')
    plt.show()
    out['run_plot_path'] = str(run_path)
    out['docs_plot_path'] = str(docs_path) if docs_path is not None else None

print(json.dumps(out, indent=2, sort_keys=True))


## Plot

<img src="../artifacts/tutorial_toi-5807-incremental/18_v16_model_competition/V16_model_competition.png" width="820" />


<details>
<summary><b>Expected Output (plot cell)</b></summary>

```text
{
  "docs_plot_path": "docs/tutorials/artifacts/tutorial_toi-5807-incremental/18_v16_model_competition/V16_model_competition.png",
  "run_plot_path": "persistent_cache/tutorial_toi-5807-incremental/18_v16_model_competition/V16_model_competition.png",
  "status": "ok"
}
```

</details>


<details>
<summary><b>Analysis</b></summary>

- **Flags:** `MODEL_PREFERS_NON_TRANSIT`.
- **Result:** best model is `transit_sinusoid` (label `SINUSOID`) with ΔBIC ≈ 11.19 vs the next-best.
- **Why this is useful:** if a non-transit component explains the data materially better, it’s a strong warning that the apparent transit may be riding on systematics or stellar variability.
- **Interpretation:** treat this as a high-priority caution; do not conclude “planet-like” purely from shape checks.
- **Next step:** V17 to probe ephemeris reliability and whether the detection is uniquely tied to the quoted period/t0.

</details>
